# Flattening Large JSON documents using Pandas

JSON data format is widely used for storing unstructured data. NoSQL databases such as MongoDB or Dynamo DB store the data in JSON format. JSON documents can be heavily nested making it difficult for Data Scientists and Analysts to extract the required information from these files in a structured manner. JSON normalization is an approach for converting these nested JSON to structured data.

This notebook shows an approach for normalizing complex JSON documents using Pandas library. Pandas is one of the most popular libraries in Python. It is extensively used for processing data in a highly efficient manner.

## Use Case

The primary use case is when we want to insert json data into relational tables.

##### We need pandas and json libraries for this to work

In [1]:
import pandas as pd
import json
import os

Let's start with a simple json without having too much nesting. These types can be falttened with a `df.json_normalize()`  function call. However arrays are not flattened automatically and they persist as lists. In order to seperate out arrays we can call `df.explode()`. `df.explode()` will break arrays into individual rows 

### SIMPLE JSON

In [2]:
data = {
    "person": {
        "First name": "Marianne",
        "Last name": "Klein",
        "age": 28,
        "address": [
            {
                "street": "861 Conroy Passage",
                "city": "Williamsfield"
            },
            {
                "street": "863 Conroy Passage",
                "city": "Amsterdam"
            }
        ]
    }
}

The function `json_normalize` flattens the json dictionary into a dataframe

In [3]:
df = pd.json_normalize(data)

Running the cell below allows to see the output. We can see how the nested structure at the first level has been broken down to sub fields having a (.) notation.

However the person.address could not be broken down as it contained a list. We will further process the list in the below steps

In [4]:
df

,person.First name,person.Last name,person.age,person.address
0,Marianne,Klein,28,"[{'street': '861 Conroy Passage', 'city': 'Wil..."


`explode()` will take the array column as parameter and return a df with each array element in seperate rows. We need to do `reset_index(drop=True)` after `explode()` because exploding will introduce a new index after the operation. We do reset to switch back to the default index  

In [5]:
df=df.explode('person.address').reset_index(drop=True)

Output from the cell below shows how each element of the list has been broken down as separate rows. However the column `person.address` is still a nested dictionary. We will process it further below

In [6]:
df

,person.First name,person.Last name,person.age,person.address
0,Marianne,Klein,28,"{'street': '861 Conroy Passage', 'city': 'Will..."
1,Marianne,Klein,28,"{'street': '863 Conroy Passage', 'city': 'Amst..."


Now we normalize `person.address` to get a clean dataframe without any nesting

In [7]:
address_df=pd.json_normalize(df['person.address'])

In [8]:
address_df

,street,city
0,861 Conroy Passage,Williamsfield
1,863 Conroy Passage,Amsterdam


In [9]:
df=df.drop('person.address', axis=1)

Now let's print this dataframe

In [10]:
print(df)

  person.First name person.Last name  person.age
0          Marianne            Klein          28
1          Marianne            Klein          28


Let's test it out with a complex json. In this case we need to make more than one csv from the available data. So we seperate out the required data and process individually

### HIGHLY COMPLEX JSON

In [11]:
data = [
  {
    "_id": "62f0b6996b5ffba02140972b",
    "index": 0,
    "guid": "c6969988-3028-4533-a0d3-19e0bc2d2895",
    "isActive": "true",
    "balance": "$1,719.20",
    "picture": "http://placehold.it/32x32",
    "age": 40,
    "eyeColor": "green",
    "name": "Jordan Allen",
    "gender": "female",
    "company": "FUTURIS",
    "email": "jordanallen@futuris.com",
    "phone": "+1 (912) 514-3472",
    "address": "868 Riverdale Avenue, Robinette, Kentucky, 1463",
    "about": "Tempor magna sunt amet exercitation magna. Aute cupidatat adipisicing ad dolor cupidatat culpa. Ipsum minim labore amet voluptate veniam aute pariatur ullamco Lorem voluptate consequat magna sunt dolor. Proident nostrud sit dolor nisi est cupidatat ad culpa quis Lorem tempor id eiusmod. Deserunt et ut ullamco reprehenderit officia. Incididunt eu duis amet in enim. Exercitation Lorem voluptate veniam ipsum qui reprehenderit tempor nulla sit.\r\n",
    "registered": "2019-08-26T06:25:06 -06:-30",
    "latitude": 73.069673,
    "longitude": 137.207595,
    "tags": [
      "non",
      "mollit",
      "velit",
      "laborum",
      "qui",
      "ipsum",
      "sint"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Jimmie Miles"
      },
      {
        "id": 1,
        "name": "Stacy Holt"
      },
      {
        "id": 2,
        "name": "Michael Larson"
      }
    ],
    "greeting": "Hello, Jordan Allen! You have 10 unread messages.",
    "favoriteFruit": "banana"
  },
  {
    "_id": "62f0b69977affbec511ebe04",
    "index": 1,
    "guid": "e5384a44-8153-404b-9ec8-7d24ab2e22c6",
    "isActive": "false",
    "balance": "$2,705.88",
    "picture": "http://placehold.it/32x32",
    "age": 34,
    "eyeColor": "brown",
    "name": "Courtney Mendoza",
    "gender": "female",
    "company": "DEVILTOE",
    "email": "courtneymendoza@deviltoe.com",
    "phone": "+1 (975) 547-2209",
    "address": "268 Engert Avenue, Jugtown, Virgin Islands, 1655",
    "about": "Veniam id amet est esse irure ad veniam non. Aliqua tempor adipisicing qui excepteur magna consequat sit incididunt est. Id esse occaecat exercitation excepteur. Dolore quis est voluptate Lorem cillum.\r\n",
    "registered": "2020-07-05T06:27:06 -06:-30",
    "latitude": 45.191384,
    "longitude": 165.063785,
    "tags": [
      "minim",
      "ullamco",
      "ipsum",
      "aliqua",
      "Lorem",
      "tempor",
      "elit"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Kim Grant"
      },
      {
        "id": 1,
        "name": "Sykes Dale"
      },
      {
        "id": 2,
        "name": "Combs Logan"
      }
    ],
    "greeting": "Hello, Courtney Mendoza! You have 10 unread messages.",
    "favoriteFruit": "apple"
  },
  {
    "_id": "62f0b699828121aa11c708ef",
    "index": 2,
    "guid": "b8c74f73-a553-4100-8705-85044594b14f",
    "isActive": "false",
    "balance": "$1,085.46",
    "picture": "http://placehold.it/32x32",
    "age": 25,
    "eyeColor": "green",
    "name": "Gates Rivas",
    "gender": "male",
    "company": "HOMETOWN",
    "email": "gatesrivas@hometown.com",
    "phone": "+1 (807) 586-3429",
    "address": "567 Colonial Road, Coldiron, Puerto Rico, 6129",
    "about": "Sint velit velit nostrud excepteur ullamco. Aliqua fugiat consectetur incididunt laboris et nulla nulla. Labore tempor incididunt tempor ut.\r\n",
    "registered": "2018-11-13T03:18:38 -06:-30",
    "latitude": -57.671522,
    "longitude": 97.722594,
    "tags": [
      "proident",
      "esse",
      "reprehenderit",
      "reprehenderit",
      "aliquip",
      "veniam",
      "ea"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Goodwin Randolph"
      },
      {
        "id": 1,
        "name": "Gibson George"
      },
      {
        "id": 2,
        "name": "Mosley Francis"
      }
    ],
    "greeting": "Hello, Gates Rivas! You have 4 unread messages.",
    "favoriteFruit": "banana"
  },
  {
    "_id": "62f0b699e4f1537974577cbe",
    "index": 3,
    "guid": "5202e6fd-c66c-4c02-90ae-edbb3e96ac6d",
    "isActive": "false",
    "balance": "$1,118.10",
    "picture": "http://placehold.it/32x32",
    "age": 38,
    "eyeColor": "brown",
    "name": "Lopez Donaldson",
    "gender": "male",
    "company": "MUSIX",
    "email": "lopezdonaldson@musix.com",
    "phone": "+1 (859) 552-2093",
    "address": "631 Clifton Place, Baden, New Mexico, 7208",
    "about": "Deserunt laboris proident est ut aliqua incididunt et fugiat ad nostrud eiusmod cillum laborum mollit. Sit ut eiusmod incididunt do id. Irure ut id in pariatur excepteur ipsum incididunt excepteur. Exercitation officia dolore velit magna tempor deserunt esse.\r\n",
    "registered": "2018-03-03T04:14:38 -06:-30",
    "latitude": 43.026358,
    "longitude": 83.525254,
    "tags": [
      "ex",
      "labore",
      "officia",
      "magna",
      "tempor",
      "dolore",
      "consectetur"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Rosario Key"
      },
      {
        "id": 1,
        "name": "Olson Long"
      },
      {
        "id": 2,
        "name": "Gamble Burke"
      }
    ],
    "greeting": "Hello, Lopez Donaldson! You have 6 unread messages.",
    "favoriteFruit": "banana"
  },
  {
    "_id": "62f0b699e7ef05cedbaf24ae",
    "index": 4,
    "guid": "5c9a9199-3ed2-40ed-bfd6-123f0045c536",
    "isActive": "true",
    "balance": "$3,203.61",
    "picture": "http://placehold.it/32x32",
    "age": 35,
    "eyeColor": "green",
    "name": "Gillespie Blankenship",
    "gender": "male",
    "company": "BEDLAM",
    "email": "gillespieblankenship@bedlam.com",
    "phone": "+1 (838) 510-2864",
    "address": "451 Seagate Avenue, Beyerville, North Carolina, 7135",
    "about": "Ad proident laborum ut cillum laboris elit ex fugiat id. Nostrud sit aliquip qui amet occaecat qui nisi ut. Excepteur nostrud irure dolor ex magna dolor anim incididunt sit cupidatat consectetur esse officia.\r\n",
    "registered": "2020-08-16T02:00:49 -06:-30",
    "latitude": -49.601916,
    "longitude": 21.588797,
    "tags": [
      "duis",
      "amet",
      "magna",
      "aute",
      "qui",
      "aliquip",
      "sunt"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Kayla Wood"
      },
      {
        "id": 1,
        "name": "Shawna Russo"
      },
      {
        "id": 2,
        "name": "Lorraine Cannon"
      }
    ],
    "greeting": "Hello, Gillespie Blankenship! You have 8 unread messages.",
    "favoriteFruit": "apple"
  },
  {
    "_id": "62f0b699016a3560b0a33a0f",
    "index": 5,
    "guid": "d0a944c4-9c2b-4aa2-a748-1b4dd9464bee",
    "isActive": "true",
    "balance": "$2,164.61",
    "picture": "http://placehold.it/32x32",
    "age": 26,
    "eyeColor": "green",
    "name": "Mccullough Fulton",
    "gender": "male",
    "company": "PROWASTE",
    "email": "mcculloughfulton@prowaste.com",
    "phone": "+1 (801) 533-3738",
    "address": "598 Kensington Walk, Harrodsburg, South Dakota, 7780",
    "about": "Ipsum culpa labore dolore aliquip sunt ex deserunt excepteur enim cupidatat voluptate eiusmod incididunt. Dolor mollit eiusmod in culpa. Excepteur aliqua cillum culpa amet ex mollit voluptate ipsum sit. Officia ipsum culpa nostrud nostrud anim exercitation minim consectetur consectetur mollit excepteur pariatur.\r\n",
    "registered": "2017-03-15T04:09:14 -06:-30",
    "latitude": -19.919437,
    "longitude": -119.567468,
    "tags": [
      "enim",
      "occaecat",
      "duis",
      "cillum",
      "dolore",
      "id",
      "quis"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Gabrielle Mckee"
      },
      {
        "id": 1,
        "name": "Summers Gross"
      },
      {
        "id": 2,
        "name": "Ellison Melendez"
      }
    ],
    "greeting": "Hello, Mccullough Fulton! You have 10 unread messages.",
    "favoriteFruit": "strawberry"
  },
  {
    "_id": "62f0b6992ef337d7b0f9c25e",
    "index": 6,
    "guid": "958bace7-72b7-4962-b6d0-2fa1e01b9879",
    "isActive": "false",
    "balance": "$2,602.72",
    "picture": "http://placehold.it/32x32",
    "age": 23,
    "eyeColor": "blue",
    "name": "Rachelle Lee",
    "gender": "female",
    "company": "EQUITAX",
    "email": "rachellelee@equitax.com",
    "phone": "+1 (815) 433-2818",
    "address": "537 Hope Street, Avalon, Federated States Of Micronesia, 1678",
    "about": "Anim velit occaecat laboris nulla minim pariatur. Ex dolore tempor est aliquip. Sunt sunt tempor fugiat officia excepteur quis esse commodo labore. Anim velit commodo irure cupidatat ullamco. Commodo sunt anim adipisicing amet eiusmod fugiat enim irure ad est. Reprehenderit dolore irure veniam aute et. Anim est eiusmod ea do.\r\n",
    "registered": "2018-01-26T02:48:34 -06:-30",
    "latitude": 43.052351,
    "longitude": -73.636871,
    "tags": [
      "ullamco",
      "officia",
      "veniam",
      "do",
      "enim",
      "sunt",
      "ullamco"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Melisa Morgan"
      },
      {
        "id": 1,
        "name": "Jacobs Cross"
      },
      {
        "id": 2,
        "name": "Margaret Frazier"
      }
    ],
    "greeting": "Hello, Rachelle Lee! You have 4 unread messages.",
    "favoriteFruit": "apple"
  }
]

The `normalize_friends` function will flatten the `friends` section of the json and return a df containing the rows

In [12]:
def normalize_friends(dat):
    new_df = pd.json_normalize(dat['friends'])
    new_df['friend_of']=dat['name']
    required_cols = ['name', 'friend_of']
    return new_df[required_cols]

The `normalize_user()` function will flatten the `user` section of the json and return a df containing the rows. We kept the required columns in a list and passed it to `json_normalize()`. As seen here `json_normalize()` can also take an array as input. This comes in handy when we want to supply the required columns dynamically

When we normalize a JSON, the JSON may have some extra columns that we are not interested in. That is why, it is always return only the *required columns*.

In [13]:
def normalize_user(dat):
    reqd_cols = ['guid','isActive','balance','age','name','gender','company','email', 'phone','address']
    df = pd.json_normalize(dat)
    df = df[reqd_cols].reset_index(drop=True)
    return df

Finally we combine the data and save them to csv files. Instead of saving to csv we can also save it directly in `DB` using some `connector`. 

If there is an array of nested JSONs, we iterate through each element of the array and concat to a Pandas DataFrame. The following example demonstrates this.

In [14]:
friends_df = pd.DataFrame([])
user_df = pd.DataFrame([])
for dat in data:
    friends_df_new = normalize_friends(dat)
    user_df_new = normalize_user(dat)
    friends_df = pd.concat([friends_df, friends_df_new]).reset_index(drop=True)
    user_df = pd.concat([user_df, user_df_new]).reset_index(drop=True)

In [15]:
friends_df

,name,friend_of
0,Jimmie Miles,Jordan Allen
1,Stacy Holt,Jordan Allen
2,Michael Larson,Jordan Allen
3,Kim Grant,Courtney Mendoza
4,Sykes Dale,Courtney Mendoza
5,Combs Logan,Courtney Mendoza
6,Goodwin Randolph,Gates Rivas
7,Gibson George,Gates Rivas
8,Mosley Francis,Gates Rivas
9,Rosario Key,Lopez Donaldson


In [16]:
user_df

,guid,isActive,balance,age,name,gender,company,email,phone,address
0,c6969988-3028-4533-a0d3-19e0bc2d2895,true,"$1,719.20",40,Jordan Allen,female,FUTURIS,jordanallen@futuris.com,+1 (912) 514-3472,"868 Riverdale Avenue, Robinette, Kentucky, 1463"
1,e5384a44-8153-404b-9ec8-7d24ab2e22c6,false,"$2,705.88",34,Courtney Mendoza,female,DEVILTOE,courtneymendoza@deviltoe.com,+1 (975) 547-2209,"268 Engert Avenue, Jugtown, Virgin Islands, 1655"
2,b8c74f73-a553-4100-8705-85044594b14f,false,"$1,085.46",25,Gates Rivas,male,HOMETOWN,gatesrivas@hometown.com,+1 (807) 586-3429,"567 Colonial Road, Coldiron, Puerto Rico, 6129"
3,5202e6fd-c66c-4c02-90ae-edbb3e96ac6d,false,"$1,118.10",38,Lopez Donaldson,male,MUSIX,lopezdonaldson@musix.com,+1 (859) 552-2093,"631 Clifton Place, Baden, New Mexico, 7208"
4,5c9a9199-3ed2-40ed-bfd6-123f0045c536,true,"$3,203.61",35,Gillespie Blankenship,male,BEDLAM,gillespieblankenship@bedlam.com,+1 (838) 510-2864,"451 Seagate Avenue, Beyerville, North Carolina..."
5,d0a944c4-9c2b-4aa2-a748-1b4dd9464bee,true,"$2,164.61",26,Mccullough Fulton,male,PROWASTE,mcculloughfulton@prowaste.com,+1 (801) 533-3738,"598 Kensington Walk, Harrodsburg, South Dakota..."
6,958bace7-72b7-4962-b6d0-2fa1e01b9879,false,"$2,602.72",23,Rachelle Lee,female,EQUITAX,rachellelee@equitax.com,+1 (815) 433-2818,"537 Hope Street, Avalon, Federated States Of M..."
